# Introduction

In general, we would like to make use of all the knowledge available in different data sources and also use the algorithmic power of various recommender systems to make robust inferences - **Hybrid Recommender Systems**. 

Three primary ways of creating hybrid recommender systems:
1. **Ensemble design**: Results from off-the-shelf algorithms are combined into a single and more robust output.
2. **Monolithic design**: An integrated recommendation algorithm is created by using various data types.
3. **Mixed systems**: These systems use multiple recommendation algorithms as black-boxes, but the items recommended by the various systems are presented togetther side by side.

The **hybrid system** is used in a broader context than the term **ensemble system**.

The **hybrid recommendation systems** can be classified into the following categories:
1. **Weighted**: The scores of several recommender systems are combined into a sigle unified score by computing the weighted aggregates of the scores from invidual ensemble components.
2. **Switching**: The algorithm switches between various recommender systems depending on current needs.
3. **Cascade**: One recommender system refines the recommendations given by another.
4. **Feature augumentation**: The output of one recommender system is used to create input features for the next.
5. **Feature combination**: The features from different data sources are combined and used in the contexr of a single recommender system.
6. **Meta-level**: The model used by one recommender system is used as input to another system. Example: Content-based system creates peer groups, then the collaborative filtering system use that peer groups to make the recommendations.
7. **Mixed**: Recommendations from several engines are presented to the user at the same time.

The first four are ensemble systems, the next two are monolithic systems, and the last one is mixed system.

# Ensemble Methods

The error of a classifier in predicting the dependent variable can be decomposed into three components:
1. **Bias**: Every classifier makes its own modeling assumptions about the nature of the decision boundary between classes. When a classifier has high bias, it will make consistently incorrect predictions over particular choices of test instances near the incorrectly modeled-decision boundary.
2. **Variance**: Random variations in the choices of the training data will lead to different models. As a result, the dependent variable for a test instance might be inconsistently predicted by different choices of training data sets. Model variance is closely related to overfitting.
3. **Noise**: The noise refers to the intrinsic errors in the target class labeling.

The expected mean-squared error of a classifier over a set of test instances can be shown to be sum of the bias, variance, and noise:
$$ Error = Bias^2 + Variance + Noise $$

Classification ensemble methods such as *bagging* reduce the variance, whereas methods such as *boosting* can reduce the bias.

# Weighted Hybrid

Let $R = [r_{uj}]$ be an $m \times n$ ratings matrix. Let $\hat{R}_1...\hat{R}_q$ be the $m \times n$
completely specified ratings matrices, in which the unobserved entries of $R$ are predicted by $q$ different algorithms. Then, for a set of weights $\alpha_1...\alpha_q$ , the weighted hybrid creates a combined prediction matrix $\hat{R} = [\hat{r}_{uj}]$ as follows:
$$ \hat{R} = \sum_{i=1}^q \alpha_i \hat{R}_i $$

In [78]:
import numpy as np
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
def ensemble_all_predicted_ratings_matrix(list_matrixs, list_weights):
    z = np.multiply(list_weights, np.transpose(list_matrix))
    z = np.transpose(z)
    
    return np.sum(z, axis = 0)/np.sum(list_weights)

In order to determine the optimal weights, it is necessary to be able to evaluate the
effectiveness of a particular combination of weights $\alpha_1...\alpha_q$. <br>
A simple approach is to hold out a small fraction (e.g., 25%) of the known entries in the $m \times n$ ratings matrix $R = [r_{uj}]$ and create the prediction matrices $\hat{R}_1...\hat{R}_q$ by applying the $q$ different base algorithms on the remaining 75% of the entries in R. The resulting predictions $\hat{R}_1...\hat{R}_q$ are then combined to create the ensemble-based prediction $\hat{R}$. Let the user-item indices $(u,j)$ of these held-out entries be denoted by $H$. The effectiveness of a particular scheme can be evaluated using either the mean-squared error (MSE) or the mean absolute error (MAE) of the predicted matrix over the held-out ratings in $H$. We can use linear regression model to find the most effective values of weights.

After the weights have been learned using linear regression, the individual component models are retrained on the entire training set without any held-out entries.

Regularization can be added to prevent overfitting. It is also possible to add other con- straints on the various values of αi such as non-negativity or ensuring that they sum to 1. 

Often, these tech- niques use a simple average of the predictions of different components. It is particularly important to weight the different components when the predicted utility values are on dif- ferent scales, or when some of the ensemble components are much more accurate than others. 

In [32]:
# indices for vector
def specified_rating_indices(u):
    return np.where(np.isfinite(u))

In [79]:
from sklearn.utils import shuffle

def get_batch(X, y, batch_size, iteration):
    return (X[iteration * batch_size: (iteration + 1) * batch_size], y[iteration * batch_size: (iteration + 1) * batch_size])

In [140]:
def find_weights_using_linear_regression(list_predicted_matrix, original_matrix,
                                         batch_size=2, n_epochs=10, learning_rate=0.001):
    indices = specified_rating_indices(original_matrix)

    X_train = []
    for i in range(len(list_predicted_matrix)):
        matrix = list_predicted_matrix[i]
        X_train.append(matrix[indices])
        
    X_train = np.transpose(X_train)
    y_train = original_matrix[indices]
    
    n_dimensions = len(list_predicted_matrix)
    
    # create tensorflow graph
    tf.reset_default_graph()
    
    X = tf.placeholder(tf.float32, shape=[None, n_dimensions], name='X')
    y = tf.placeholder(tf.float32, shape=None, name='y')
    
#     W = tf.get_variable('W', shape=[n_dimensions, 1], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
#     b = tf.Variable(dtype=tf.float32, initial_value=0)
#     output = tf.matmul(X, W) + b

    output = tf.layers.dense(X, units=1)
    
    loss = tf.losses.mean_squared_error(output, y)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        init.run()

        for epoch in range(n_epochs):
            X_train, y_train = shuffle(X_train, y_train)
            total_loss = 0
            n_iteration = X_train.shape[0] // batch_size
            for i in range(n_iteration):
                X_batch, y_batch = get_batch(X_train, y_train, batch_size, i)
                l, _ = sess.run([loss, optimizer], feed_dict={X: X_batch, y: y_batch})
                total_loss += l
            
            print('Loss: ', total_loss / n_iteration)
        
        var = [v for v in tf.trainable_variables()][0]
        value = sess.run(var)
    return value

In [156]:
a = np.array([[1, 2, 9, 4, 5],
              [nan, 1, 3, nan, 2]])
b = np.array([[2, 2, 4, 5, 6],
              [nan, 1, 3, nan, 2]])
c = np.array([[3, 4, 5, 6, 7],
              [nan, 1, 3, nan, 2]])

value = find_weights_using_linear_regression([b, c], a, n_epochs=10)

Loss:  19.537912786006927
Loss:  15.251693665981293
Loss:  11.021173030138016
Loss:  8.004278674721718
Loss:  7.621613174676895
Loss:  7.934566304087639
Loss:  6.653058409690857
Loss:  8.155106753110886
Loss:  7.148072227835655
Loss:  6.631481006741524


In [157]:
value

array([[ 1.0894009 ],
       [-0.20308039]], dtype=float32)

## Various Types of Model Combinations

There are typically two forms of model combinations:
1. **Homogeneous data type and model classes**: Different models are applied on the same data. Such an approach is robust because it avoids the specific bias of particular algorithms on a given data set even though all the constituent models belong to the same class
2. **Heterogeneous data type and model classes**: Different classes of models are applied to different data sources. The idea is to leverage the complementary knowledge in the various data sources in order to provide the most accurate recommendations.

## Adapting Bagging from Classification

The basic idea in bagging is to reduce the variance component of the error in classification. In bagging, $q$ training data sets are created with bootstrapped sampling. In bootstrapped sampling, rows of the data matrix are sampled with replacement in order to create a new training data set of the same size as the original training data set. This new training data set typically contains many duplicates.

A particular variant of bagging, known as subagging, subsamples the rows, rather than sampling with replacement. For example, one can simply use all the distinct rows in a bootstrapped sample for training the models.

1. **Row-wise bootstrapping**: the rows of the ratings matrix $R$ are sampled with replacement to create a new ratings matrix of the same dimensions.
2. **Row-wise subsampling**: This approach is similar to row-wise bootstrapping, except that the rows are sampled without replacement. The fraction $f$ of rows sampled is chosen randomly from $(0.1,0.5)$. The number of ensemble components q should be significantly greater than 10 to ensure that all rows are represented.
3. **Entry-wise bagging**: the entries of the original ratings matrix are sampled with replacement to create the $q$ different ratings matrices $R_1...R_q$. 
4. **Entry-wise subsampling**: In entry-wise subsampling, a fraction of the entries are retained at random from the ratings matrix R to create a sampled training data set.

In [1]:
# Example for row-wise bootstraping

In [9]:
import numpy as np

In [8]:
def create_variant_matrix(ratings_matrix, q):
    shape = ratings_matrix.shape
    n_users = shape[0]
    n_items = shape[1]
    
    list_new_matrixs = []
    list_indice_users = [] 
    # we need to store the indices because we will compute the average rating for each user
    # based on number of times that user appears in new matrix.
    
    for i in range(q):
        random_users = np.random.randint(n_users, size=[n_users])
        new_matrix = ratings_matrix[random_users]
        
        list_indice_users.append(random_users)
        list_new_matrixs.append(new_matrix)
        
    return list_new_matrixs, list_indice_users

In [31]:
def calculate_average_rating(list_predicted_matrices, list_indice_users):
    n_users = list_predicted_matrices[0].shape[0]
    n_items = list_predicted_matrices[0].shape[0]
    
    average_ratings_matrix = np.zeros(shape=[n_users, n_items])
    n_appearances = np.zeros(shape=[n_users])
    
    for i in range(len(list_predicted_matrices)):
        matrix = list_predicted_matrices[i]
        for j in range(len(list_indice_users)):
            indice = list_indice_users[j]
            average_ratings_matrix[indice] += matrix[j]
            n_appearances[indice] += 1
    
    n_appearances = n_appearances.reshape(n_users, -1)
    
    average_ratings_matrix = np.divide(average_ratings_matrix, n_appearances)
    
    return average_ratings_matrix

In [33]:
a = np.random.rand(10, 10)
matrices, indices = create_variant_matrix(a, 10)

In [34]:
calculate_average_rating(matrices, indices)

array([[0.5544684 , 0.48302639, 0.49897061, 0.45261422, 0.41146809,
        0.59407013, 0.51869078, 0.40650742, 0.43103446, 0.51774474],
       [0.57122743, 0.47417499, 0.50438522, 0.42770641, 0.46337349,
        0.56110212, 0.54255025, 0.45046273, 0.38696895, 0.49242176],
       [0.60770575, 0.47964033, 0.47405594, 0.40975171, 0.40559398,
        0.61054964, 0.54060945, 0.46732633, 0.44447129, 0.52949623],
       [0.56329272, 0.47996929, 0.48743222, 0.38879544, 0.41325252,
        0.60202025, 0.51795367, 0.50260045, 0.34654813, 0.51158225],
       [0.57629426, 0.48050737, 0.4670425 , 0.36075788, 0.39757844,
        0.60724309, 0.53803475, 0.48301275, 0.41653515, 0.46425253],
       [0.63453612, 0.50996145, 0.44856343, 0.42314875, 0.45070946,
        0.56414683, 0.59638614, 0.45267081, 0.45373092, 0.47035176],
       [0.56751797, 0.4781543 , 0.48524757, 0.37820407, 0.38739733,
        0.62766389, 0.52194064, 0.49616308, 0.40296669, 0.50343461],
       [0.54864541, 0.46263895, 0.4925356

## Randomness Injection

The basic idea is to take a base classifier and explicitly inject randomness into the classifier.
1. **Injecting randomness into a neighborhood model**: Instead of using the $top-k$ nearest neighbors (users or items) in a user-based or item-based neighborhood model, the $top-\alpha.k$ neighbors are selected for $\alpha >> 1$. Then, k elements are randomly selected from these $\alpha.k$ neighbors.
2. **Injecting randomness into a matrix factorization model**: By choosing different initializations, different solutions are often obtained. The combinations of these different solutions often provide more accurate results.

# Switching Hybrids

Switching hybrids are used most commonly in recommender systems in the context of the problem of **model selection**.

## Switching Mechanisms for Cold-Start Issues

Switching mechanisms are often used to handle the cold-start problem, in which one rec- ommender performs better with less data, whereas the other recommender performs better with more data. One might use a knowledge-based recommender, when few ratings are avail- able because knowledge-based recommender systems can function without any ratings, and they are dependent on user specifications of their needs. However, as more ratings become available, one might switch to a collaborative recommender.

## Bucket-of-Models

In this approach, a fraction (e.g., 25% to 33%) of the specified entries in the ratings matrix are held out, and various models are applied to the resulting matrix. The held-out entries are then used to evaluate the effectiveness of the model. Once the relevant model has been selected, it is retrained on the entire ratings matrix, and the results are reported.

# Cascade Hybrids

A recommender is allowed to use recommendations of the previous recommender in any way (beyond just direct refinement), and then combine the results to make the final recommendation

## Successive Refinement of Recommendations

A recommender system successively refines the output of recommendations from the previous iteration. For example, the first recommender can provide a rough rank- ing and also eliminate many of the potential items. The second level of recommendation then uses this rough ranking to further refine it and break ties. The resulting ranking is presented to the user.

## Boosting

In traditional boosting, a sequence of training rounds is used with weighted training examples. The weights in each round are modified depending on the performance of the classifier in the previous round. Specifically, the weights on the training examples with error are increased, whereas the weights on the correctly modeled examples are reduced. As a result, the classifier is biased towards correctly classifying the examples that it was unable to properly classify in the previous round. By using several such rounds, one obtains a sequence of classification models. For a given test instance, all models are applied to it, and the weighted prediction is reported as the relevant one.

# Feature Augmentation Hybrids

The feature augmentation hybrid shares a number of intuitive similarities with the stacking ensemble in classification. In stacking, the first level classifier is used to create or aug- ment a set of features for the second level classifier.<br>

Instead of using a collaborative system first, it is also possible to use the content-based system first. The basic idea is to use the content-based system to fill in the missing entries of the ratings matrix so that it is no longer sparse. Thus, the missing entries are estimated by the content-based system to create a denser ratings matrix.

Then, a collaborative recommender is used on the dense ratings matrix to make rating predictions.

# Meta-Level Hybrids

In a meta-level hybrid, the model learned by one recommender is used as input to the next level.

The main idea here is that the content-based peer group is used to determine the most similar users of the target user. Once the peer group has been determined, then the weighted average of the ratings of the peer group are used to determine the predicted ratings.

# Feature Combination Hybrids

In feature combination hybrids, the idea is to combine the input data from various sources (e.g., content and collaborative) into a unified representation before applying a predictive algorithm. In most cases, this predictive algorithm is a content-based algorithm that uses collaborative information as additional features.

Another approach is to augment a ratings matrix and add columns for keywords in addition to items. Therefore, the ratings matrix becomes an $m \times (n + d)$ matrix, where n is the number of items and $d$ is the number of keywords. The weights of “keyword items” are based on the weighted aggregation of the descriptions of the items accessed, bought, or rated by the user. A traditional neighborhood or matrix factorization approach can be used with this augmented matrix. The relative weights of the two types of columns can be learned through cross-validation. This type of combination of two optimization models is common in hybrid settings, where the objective function is set up as follows in terms of a parameter vector $\hat{\theta}$:
$$J = CollaborativeObjective(\hat{\theta}) + \beta ContentObjective(\hat{\theta}) + Regularization$$


## Regression and Matrix Factorization

$$ Minimize\ J = \|R − RW\|^2 + \beta · \|R − CW\|^2 + \lambda\|W\|^2 + \lambda_1 · \|W|_1 $$ $$ subject to:$$
$$ W≥0$$
$$ Diagonal(W ) = 0 $$

## Meta-level Features

New meta-features can be extracted from features of a particular type of recommender and then combined within the ensemble model. For example, one can extract meta-level features from a ratings matrix corresponding to the number of ratings given by various users and items. 

# Mixed Hybrids

The main characteristic of mixed recommender systems is that they combine the scores from different components in terms of presentation, rather than in terms of combining the predicted scores.